In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
from dotenv import load_dotenv

True

In [11]:
load_dotenv()
GGDEALS_API_KEY = os.getenv('GGDEALS_API_KEY')
if not GGDEALS_API_KEY:
    raise ValueError("Please set the GGDEALS_API_KEY environment variable.")

In [14]:
STEAM_APP_ID = 812140  # Example: Assassin's Creed Odyssey

In [29]:
url = f"http://api.gg.deals/v1/prices/by-steam-app-id/?ids={STEAM_APP_ID}&key={GGDEALS_API_KEY}&region=eu"

In [30]:
try:
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        print("Data fetched successfully!")
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

except Exception as e:
    print(f"An error occurred: {e}")

Data fetched successfully!


In [26]:
data

{'success': True,
 'data': {'812140': {'title': "Assassin's Creed Odyssey",
   'url': 'https://gg.deals/game/assassin-s-creed-odyssey/',
   'prices': {'currentRetail': '8.99',
    'currentKeyshops': '7.19',
    'historicalRetail': '4.80',
    'historicalKeyshops': '6.54',
    'currency': 'EUR'}}}}

#### Loading data from ITAD API

In [51]:
load_dotenv()
ITAD_API_KEY = os.getenv('ITAD_API_KEY')
if not ITAD_API_KEY:
    raise ValueError("Please set the ITAD_API_KEY environment variable.")

In [52]:
url_appID = "https://api.isthereanydeal.com/lookup/id/shop/61/v1"

payload = [f"app/{STEAM_APP_ID}"] # this API accepts a list of identifiers

headers = {
    "Authorization": f"key {ITAD_API_KEY}",
    "Content-Type": "application/json"
}

In [53]:
try:
    resp = requests.post(url_appID, headers=headers, json=payload, timeout=15)
    resp.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print("HTTP error:", http_err, "| Body:", getattr(resp, "text", ""))
except requests.exceptions.RequestException as req_err:
    print("Request error:", req_err)
else:
    game_id = resp.json()
    print("Raw JSON:", resp.json())

Raw JSON: {'app/812140': '018d937f-0184-7248-8d64-3c723c523111'}


In [54]:
ITAD_GAME_ID = game_id['app/812140']
print("ITAD Game ID:", ITAD_GAME_ID)

ITAD Game ID: 018d937f-0184-7248-8d64-3c723c523111


In [59]:
url_info = f"https://api.isthereanydeal.com/games/info/v2"

params2 = {"id": ITAD_GAME_ID, "key": ITAD_API_KEY}

In [66]:
try:
    r = requests.get(url_info, params=params2, timeout=15)
    r.raise_for_status()
    info = r.json()
    print("Game title:", info.get("title"))
    print("Tags:", info.get("tags"))
    print("Release date:", info.get("releaseDate"))
    print("Publisher:", info.get("publishers"))
    print("Players:", info.get("players"))
    print("Reviews:", info.get("reviews"))
except requests.HTTPError as e:
    print("HTTP error:", e, "| Body:", r.text)

Game title: Assassin's Creed Odyssey
Tags: ['Open World', 'RPG', 'Singleplayer', 'Historical', 'Action']
Release date: 2018-10-03
Publisher: [{'id': 61, 'name': 'Ubisoft'}]
Players: {'recent': 5162, 'day': 7903, 'week': 8770, 'peak': 61984}
Reviews: [{'score': 89, 'source': 'Steam', 'count': 158914, 'url': 'https://store.steampowered.com/app/812140/'}, {'score': 86, 'source': 'Metascore', 'count': 14, 'url': 'https://metacritic.com/game/assassins-creed-odyssey/critic-reviews/?platform=pc'}, {'score': 69, 'source': 'Metacritic User Score', 'count': 5304, 'url': 'https://metacritic.com/game/assassins-creed-odyssey/user-reviews/?platform=pc'}, {'score': 84, 'source': 'OpenCritic', 'count': 169, 'url': 'https://opencritic.com/game/6222/assassins-creed-odyssey'}]
